In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/rps.zip \
    -O /tmp/rps.zip
  
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/rps-test-set.zip \
    -O /tmp/rps-test-set.zip

/bin/bash: /home/danieldu/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
wget: /home/danieldu/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-10-11 17:17:50--  https://storage.googleapis.com/learning-datasets/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.163.59, 142.251.43.27, 172.217.160.91, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.163.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/rps.zip’

/tmp/rps.zip        100%[===================>] 191.38M  27.5MB/s    in 7.2s    

2023-10-11 17:17:57 (26.6 MB/s) - ‘/tmp/rps.zip’ saved [200682221/200682221]

/bin/bash: /home/danieldu/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
wget: /home/danieldu/anaconda3/lib/libuuid.so.1: no version information available (required by wget)
--2023-1

In [2]:
import zipfile

def extract_file(local_zip,dir):
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('/tmp/')
    zip_ref.close()
    
extract_file('/tmp/rps.zip',dir)
extract_file('/tmp/rps-test-set.zip','/tmp/')

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/tmp/rps/"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "/tmp/rps-test-set/"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)



Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

2023-10-11 17:18:02.688650: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-11 17:18:02.714786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-11 17:18:02.714966: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-11 17:18:02.715532: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
history = model.fit(
  train_generator,
  epochs=100,
  steps_per_epoch=20,
  validation_data = validation_generator,
  verbose = 1,
  batch_size=64,
  validation_steps=3)

Epoch 1/100


2023-10-11 17:18:05.117687: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8500


20/20 [==============================] - 15s 632ms/step - loss: 1.2907 - accuracy: 0.3571 - val_loss: 1.0084 - val_accuracy: 0.5780
Epoch 2/100
20/20 [==============================] - 12s 619ms/step - loss: 1.3431 - accuracy: 0.3786 - val_loss: 1.0767 - val_accuracy: 0.4462
Epoch 3/100
20/20 [==============================] - 12s 616ms/step - loss: 1.1294 - accuracy: 0.4532 - val_loss: 1.0934 - val_accuracy: 0.3333
Epoch 4/100
20/20 [==============================] - 12s 608ms/step - loss: 0.9913 - accuracy: 0.4873 - val_loss: 0.6913 - val_accuracy: 0.7285
Epoch 5/100
20/20 [==============================] - 13s 630ms/step - loss: 0.8867 - accuracy: 0.6004 - val_loss: 0.5667 - val_accuracy: 0.6640
Epoch 6/100
20/20 [==============================] - 12s 620ms/step - loss: 0.7636 - accuracy: 0.6587 - val_loss: 0.7131 - val_accuracy: 0.6102
Epoch 7/100
20/20 [==============================] - 12s 618ms/step - loss: 0.6697 - accuracy: 0.7167 - val_loss: 0.2747 - val_accuracy: 0.9516
Epoc

In [ ]:
model.save("rps.h5")
